# SQL Coder-7b-2
* This notebook uses DeFog's SQL Coder.
* Defog's SQLCoder is a family of state-of-the-art LLMs for converting natural language questions to SQL queries.
* Run the cells below to run inference on our text-to-SQL LLM

* [find it on github](https://github.com/defog-ai/sqlcoder?tab=readme-ov-file)


1. Setup

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.8 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
memory_available = torch.cuda.get_device_properties(0).total_memory

In [5]:
memory_available

15835660288

# Download the Model

Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16.If unavailable use a GPU with atleast 8GB VRAM to load this in 8bit, or with minimu 5GB of VRAM to load in 4 bit.

This step can take around 5 minutes.

In [8]:
model_name = 'defog/sqlcoder-7b-2'
tokenizer=AutoTokenizer.from_pretrained(model_name)
if memory_available>16e9:
  # i.e. atleast 15GB of GPU memory
  model=AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.float16,
                                             device_map='auto',
                                             use_cache=True)
else:
  # load in 8 bits - a tad bit slower
  model=AutoModelForCausalLM.from_pretrained(model_name,
                                             trust_remote_code=True,
                                             device_map='auto',
                                             load_in_8bit=True,
                                             use_cache=True)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Set the Question & Prompt and Tokenize

You can customize the schema in the promptd

In [9]:
prompt="""### Task
Generate a SQL query to answer [QUESTION] {question} (/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -— Name of the product
  price DECIMAL (10,2), -- Price of each unit of the product
  quantity INTEGER -- Current quantity in stock
  );

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
  name VARCHAR(50), -- Name of the customer
  address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
  );

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -— Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER, -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);


CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL (10,2) -- Unit price charged by supplier
);

- sales-product_id can be joined with products-product_id
- sales.customer_id can be joined with customers. customer_id
- sales salesperson_id can be joined with salespeople.salesperson_id
- product_suppliers.product_id can be joined with products-product_id

### Answer
Given the database schema, here is the SOL query that answers [QUESTION]{question}[/QUESTION]
[SQL]"""

# Generate the SQL

This can be very slow on Colab TPUs (about 10-20s per query)

On faster GPUs it takes 1-2s ideally.

Thus, we can use `num_beams=4` for the best results. Due to constraints, we'll stick to 1 query.

In [17]:
import sqlparse

def generate_query(question):
  updated_prompt=prompt.format(question=question)
  inputs=tokenizer(updated_prompt,return_tensors='pt').to('cuda')
  generated_ids=model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      max_new_tokens=400,
      num_beams=1,
      do_sample=False
  )
  outputs=tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  query=sqlparse.format(outputs[0].split("[SQL]")[-1],reindent=True)
  return query

In [18]:
question="What was the highest quantity sold last month?"
query=generate_query(question)
print(query)


SELECT MAX(s.quantity) AS max_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month');


In [19]:
question="Which salesperson sold large amounts of products last month?"
query=generate_query(question)
print(query)


SELECT s.salesperson_id,
       SUM(s.quantity) AS total_quantity
FROM sales s
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY s.salesperson_id
ORDER BY total_quantity DESC NULLS LAST
LIMIT 1;


In [20]:
question="What was our revenue by product in the new york region last month?"
query=generate_query(question)
print(query)


SELECT s.product_id,
       SUM(s.price * s.quantity) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY s.product_id
ORDER BY revenue DESC NULLS LAST;


---